In [9]:
import utils
import sys
import numpy as np
import pandas as pd
from bm25 import BM25

In [33]:
from importlib import reload
utils = reload(utils)

In [2]:
fd = open('data/docs.tsv','rt')
index = utils.load_from_file('pkls/index.pkl')

In [26]:
title, body = utils.get_document(fd, index, 236085)

In [27]:
title

'БРАТСКИЙ ВЕСТНИК 4 1972'

In [10]:
bm25_model = BM25('some.txt')

In [11]:
bm25_model.N = 500000
inverted_terms = {'president': 40000*[1], 'linkoln': 300*[1]}
#def score_bm25_internal(self, query_terms, inverted_terms, doc_terms, doc_len, avg_doc_len, k1, b, smooth_idf=False):

query_terms = ['president', 'linkoln' ]
doc_terms = { 'president': 15, 'linkoln': 25}
doc_len = 90
avg_doc_len = 100
k1 = 1.2
b = 0.75
bm25_model.score_bm25_internal(query_terms, inverted_terms, doc_terms, doc_len, avg_doc_len, k1, b, True)

20.625189631270004

In [12]:
import nltk

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/m.plekhanov/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
from nltk.corpus import stopwords

In [17]:
stop_words = stopwords.words('russian')

In [18]:
stop_words

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [20]:
queries = utils.load_queries('data/queries_tr.tsv')
test_queries = utils.load_test('data/test.tsv')
train_queries = utils.load_train('data/clicks.train.tsv')

In [21]:
queries[0]

'1 как называется форма приведенная на рисунке фио какую организацию представляет должность дата и способ направления приглашения подтверждение получения приглашения дата согласие принять участие в работе да нет предполагает ли выступить тема выступления нуждается ли в гостинице нуждается ли в билетах на обратный путь самолет поезд и т п прибыл на заседание 1 2 3 4 5 6 7 8 9 киселева мария владимировна ао восток главный бухгалтер 12 04 14 письмо приглашение 14 04 14 да нет нет нет да'

In [29]:
import pymorphy2
import sys

cache = dict()
morph = pymorphy2.MorphAnalyzer()

def norm_word(word):
    if word in cache:
        return cache[word]
    parsed = morph.parse(word)
    if len(parsed) == 0:
        cache[word] = word
        return word
    cache[word] = parsed[0].normal_form    
    return parsed[0].normal_form

def norm_sentence(sentence):
    out = []
    for word in sentence.split(' '):
        out.append(norm_word(word))
    return " ".join(out)

In [34]:
docs = {}
with open("tmp/broken.txt","rt") as infd:
    for line in infd:
        doc_id = int(line.strip())
        title, body = utils.get_document(fd, index, doc_id)
        docs[doc_id] = (norm_sentence(title), norm_sentence(body))

In [40]:
with open("data/docs_normed.tsv","rt") as normfd, open("data/docs_normed2.tsv","wt") as outfd:
    ind = 0
    for line in normfd:
        parts = line.strip().split('\t')
        doc_id, title, body = parts
        doc_id = int(doc_id)
        while doc_id != ind:
            if ind in docs:
                print("{}\t{}\t{}".format(ind, docs[ind][0], docs[ind][1]), file=outfd)
                ind += 1
        outfd.write(line)
        ind+=1